# Recommendation System Project: IBM Community

In this notebook, you will be putting your recommendation skills to use on real data from the IBM Watson Studio platform. 


You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/rubrics/3325/view).  **Please save regularly.**

By following the table of contents, you will build out a number of different methods for making recommendations that can be used for different situations. 


## Table of Contents

I. [Exploratory Data Analysis](#Exploratory-Data-Analysis)<br>
II. [Rank Based Recommendations](#Rank)<br>
III. [User-User Based Collaborative Filtering](#User-User)<br>
IV. [Content Based Recommendations](#Content-Recs)<br>
V. [Matrix Factorization](#Matrix-Fact)<br>
VI. [Extras & Concluding](#conclusions)

At the end of the notebook, you will find directions for how to submit your work.  Let's get started by importing the necessary libraries and reading in the data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import project_tests as t


df = pd.read_csv(
    'data/user-item-interactions.csv', 
    dtype={'article_id': int, 'title': str, 'email': str}
)
# Show df to get an idea of the data
df.head()

### <a class="anchor" id="Exploratory-Data-Analysis">Part I : Exploratory Data Analysis</a>

Use the dictionary and cells below to provide some insight into the descriptive statistics of the data.

`1.` Are there any missing values? If so, provide a count of missing values. If there are missing values in `email`, assign it the same id value `"unknown_user"`.

In [ ]:
# Some interactions do not have a user associated with it, assume the same user.
df.info()

In [ ]:
print(f"Number of Null email values is: {df.email.isna().sum()}")

In [ ]:
df[df.email.isna()]

In [ ]:
# Fill email NaNs with "unknown_user"
df['email'] = df['email'].fillna('unknown_user')

In [ ]:
# Check if no more NaNs 
df[df.email.isna()]

`2.` What is the distribution of how many articles a user interacts with in the dataset?  Provide a visual and descriptive statistics to assist with giving a look at the number of times each user interacts with an article.

In [ ]:
# What are the descriptive statistics of the number of articles a user interacts with?
user_interactions = df.groupby('email').size()
print("Descriptive statistics for number of articles per user:")
print(user_interactions.describe())

In [ ]:
# Create a plot of the number of articles read by each user
user_interactions = df.groupby('email').size()
plt.figure(figsize=(12, 6))
plt.hist(user_interactions, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Number of articles')
plt.ylabel('Number of users')
plt.title('Distribution of Articles Read by Each User')
plt.show()

print(f"Users reading 1 article: {(user_interactions == 1).sum()}")
print(f"Users reading 2-5 articles: {((user_interactions >= 2) & (user_interactions <= 5)).sum()}")
print(f"Users reading more than 10 articles: {(user_interactions > 10).sum()}")

In [ ]:
# Create a plot of the number of times each article was read
article_interactions = df.groupby('article_id').size()
plt.figure(figsize=(12, 6))
plt.hist(article_interactions, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Number of users')
plt.ylabel('Number of articles')
plt.title('Distribution of Article Usage (How Many Users Read Each Article)')
plt.show()

print(f"Articles read by 1 user: {(article_interactions == 1).sum()}")
print(f"Articles read by 2-10 users: {((article_interactions >= 2) & (article_interactions <= 10)).sum()}")
print(f"Articles read by more than 50 users: {(article_interactions > 50).sum()}")

In [ ]:
# Fill in the median and maximum number of user_article interactions below
user_interactions = df.groupby('email').size()

median_val = user_interactions.median()  # 50% of individuals interact with ____ number of articles or fewer.
max_views_by_user = user_interactions.max()  # The maximum number of user-article interactions by any 1 user is ______.

print(f"Median interactions per user: {median_val}")
print(f"Maximum interactions by any user: {max_views_by_user}")

`3.` Use the cells below to find:

**a.** The number of unique articles that have an interaction with a user.  
**b.** The number of unique articles in the dataset (whether they have any interactions or not).<br>
**c.** The number of unique users in the dataset. (excluding null values) <br>
**d.** The number of user-article interactions in the dataset.

In [ ]:
unique_articles = df['article_id'].nunique()  # The number of unique articles that have at least one interaction
total_articles = df['article_id'].nunique()  # The number of unique articles on the IBM platform (same as unique_articles in this dataset)
unique_users = df['email'].nunique()  # The number of unique users
user_article_interactions = len(df)  # The number of user-article interactions

print(f"Unique articles with interactions: {unique_articles}")
print(f"Total articles on platform: {total_articles}")
print(f"Unique users: {unique_users}")
print(f"Total user-article interactions: {user_article_interactions}")

`4.` Use the cells below to find the most viewed **article_id**, as well as how often it was viewed.  After talking to the company leaders, the `email_mapper` function was deemed a reasonable way to map users to ids.  There were a small number of null values, and it was found that all of these null values likely belonged to a single user (which is how they are stored using the function below).

In [ ]:
article_views = df['article_id'].value_counts()
most_viewed_article_id = str(article_views.index[0])  # The most viewed article in the dataset as a string with one value following the decimal 
max_views = article_views.iloc[0]  # The most viewed article in the dataset was viewed how many times?

print(f"Most viewed article ID: {most_viewed_article_id}")
print(f"Most viewed article was viewed: {max_views} times")
print(f"Article title: {df[df.article_id == float(most_viewed_article_id)]['title'].iloc[0]}")

In [ ]:
## No need to change the code here - this will be helpful for later parts of the notebook
# Run this cell to map the user email to a user_id column and remove the email column

def email_mapper(df=df):
    coded_dict = {
        email: num 
        for num, email in enumerate(df['email'].unique(), start=1)
    }
    return [coded_dict[val] for val in df['email']]

df['user_id'] = email_mapper(df)
del df['email']

# show header
df.head()

In [ ]:
## If you stored all your results in the variable names above, 
## you shouldn't need to change anything in this cell

sol_1_dict = {
    '`50% of individuals have _____ or fewer interactions.`': median_val,
    '`The total number of user-article interactions in the dataset is ______.`': user_article_interactions,
    '`The maximum number of user-article interactions by any 1 user is ______.`': max_views_by_user,
    '`The most viewed article in the dataset was viewed _____ times.`': max_views,
    '`The article_id of the most viewed article is ______.`': most_viewed_article_id,
    '`The number of unique articles that have at least 1 rating ______.`': unique_articles,
    '`The number of unique users in the dataset is ______`': unique_users,
    '`The number of unique articles on the IBM platform`': total_articles
}

# Test your dictionary against the solution
t.sol_1_test(sol_1_dict)

### <a class="anchor" id="Rank">Part II: Rank-Based Recommendations</a>

In this project, we don't actually have ratings for whether a user liked an article or not.  We only know that a user has interacted with an article. In these cases, the popularity of an article can really only be based on how often an article was interacted with.

`1.` Fill in the function below to return the **n** top articles ordered with most interactions as the top. Test your function using the tests below.

In [ ]:
def get_top_articles(n, df=df):
    """
    INPUT:
    n - (int) the number of top articles to return
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    top_articles - (list) A list of the top 'n' article titles 
    
    """
    # Count interactions per article and get top n
    article_counts = df['article_id'].value_counts()
    top_article_ids = article_counts.head(n).index.tolist()
    
    # Get corresponding titles
    top_articles = []
    for article_id in top_article_ids:
        title = df[df['article_id'] == article_id]['title'].iloc[0]
        top_articles.append(title)
    
    return top_articles # Return the top article titles from df

def get_top_article_ids(n, df=df):
    """
    INPUT:
    n - (int) the number of top articles to return
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    top_articles - (list) A list of the top 'n' article ids 
    
    """
    # Count interactions per article and get top n article IDs
    article_counts = df['article_id'].value_counts()
    top_articles = article_counts.head(n).index.tolist()
    
    return top_articles # Return the top article ids

In [ ]:
print(get_top_articles(10))
print(get_top_article_ids(10))

In [ ]:
# Test your function by returning the top 5, 10, and 20 articles
top_5 = get_top_articles(5)
top_10 = get_top_articles(10)
top_20 = get_top_articles(20)

# Test each of your three lists from above
t.sol_2_test(get_top_articles)

### <a class="anchor" id="User-User">Part III: User-User Based Collaborative Filtering</a>


`1.` Use the function below to reformat the **df** dataframe to be shaped with users as the rows and articles as the columns.  

* Each **user** should only appear in each **row** once.


* Each **article** should only show up in one **column**.  


* **If a user has interacted with an article, then place a 1 where the user-row meets for that article-column**.  It does not matter how many times a user has interacted with the article, all entries where a user has interacted with an article should be a 1.  


* **If a user has not interacted with an item, then place a zero where the user-row meets for that article-column**. 

Use the tests to make sure the basic structure of your matrix matches what is expected by the solution.

In [ ]:
# create the user-article matrix with 1's and 0's

def create_user_item_matrix(df, fill_value=0):
    """
    INPUT:
    df - pandas dataframe with article_id, title, user_id columns
    
    OUTPUT:
    user_item - user item matrix 
    
    Description:
    Return a matrix with user ids as rows and article ids on the columns with 1 values where a user interacted with 
    an article and a 0 otherwise
    """
    # Create a DataFrame with user_id, article_id, and interaction (1)
    df_interactions = df[['user_id', 'article_id']].drop_duplicates()
    df_interactions['interaction'] = 1
    
    # Create pivot table with user_id as rows, article_id as columns
    user_item = df_interactions.pivot_table(index='user_id', 
                                           columns='article_id', 
                                           values='interaction', 
                                           fill_value=fill_value)
    
    return user_item # return the user_item matrix 

user_item = create_user_item_matrix(df)

In [ ]:
## Tests: You should just need to run this cell.  Don't change the code.
assert user_item.shape[0] == 5149, "Oops!  The number of users in the user-article matrix doesn't look right."
assert user_item.shape[1] == 714, "Oops!  The number of articles in the user-article matrix doesn't look right."
assert user_item.sum(axis=1)[1] == 36, "Oops!  The number of articles seen by user 1 doesn't look right."
print("You have passed our quick tests!  Please proceed!")

`2.` Complete the function below which should take a user_id and provide an ordered list of the most similar users to that user (from most similar to least similar).  The returned result should not contain the provided user_id, as we know that each user is similar to him/herself. Because the results for each user here are binary, it (perhaps) makes sense to compute similarity as the dot product of two users. 

Use the tests to test your function.

In [ ]:
# Lets use the cosine_similarity function from sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def find_similar_users(user_id, user_item=user_item, include_similarity=False):
    """
    INPUT:
    user_id - (int) a user_id
    user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    include_similarity - (bool) whether to include the similarity in the output
    
    OUTPUT:
    similar_users - (list) an ordered list where the closest users (largest dot product users)
                    are listed first
    
    Description:
    Computes the similarity of every pair of users based on the dot product
    Returns an ordered list of user ids. If include_similarity is True, returns a list of lists
    where the first element is the user id and the second the similarity.
    
    """
    
    # compute similarity of each user to the provided user using cosine similarity
    user_similarities = cosine_similarity(user_item.loc[[user_id]], user_item)[0]

    # sort by similarity (descending order)
    user_indices = user_item.index.tolist()
    similarity_pairs = list(zip(user_indices, user_similarities))
    similarity_pairs.sort(key=lambda x: x[1], reverse=True)
    
    # remove the own user's id
    similarity_pairs = [pair for pair in similarity_pairs if pair[0] != user_id]
    
    # create list of just the ids
    most_similar_users = [pair[0] for pair in similarity_pairs]
    
    # create list of just the similarities  
    similarities = [pair[1] for pair in similarity_pairs]
    
    if include_similarity:
        return [[user_id, sim] for user_id, sim in similarity_pairs] # return a list of lists with [[similar_user_id, similarity], ...]
    return most_similar_users # return a list of the users in order from most to least similar
        

In [ ]:
# Do a spot check of your function
print("The 10 most similar users to user 1 are: {}".format(find_similar_users(1)[:10]))
print("The 5 most similar users to user 3933 are: {}".format(find_similar_users(3933)[:5]))
print("The 3 most similar users to user 46 are: {}".format(find_similar_users(46)[:3]))

`3.` Now that you have a function that provides the most similar users to each user, you will want to use these users to find articles you can recommend.  Complete the functions below to return the articles you would recommend to each user. 

In [ ]:
def get_article_names(article_ids, df=df):
    """
    INPUT:
    article_ids - (list) a list of article ids
    df - (pandas dataframe) df as defined at the top of the notebook
    
    OUTPUT:
    article_names - (list) a list of article names associated with the list of article ids 
                    (this is identified by the title column in df)
    """
    article_names = []
    for article_id in article_ids:
        title = df[df['article_id'] == article_id]['title'].iloc[0]
        article_names.append(title)
    
    return article_names # Return the article names associated with list of article ids

def get_ranked_article_unique_counts(article_ids, user_item=user_item):
    """
    INPUT:
    article_ids - (list) a list of article ids
    user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise 
    
    OUTPUT:
    ranked_article_unique_counts - (list) a list of tuples with article_id and number of 
                     unique users that have interacted with the article, sorted
                     by the number of unique users in descending order
    
    Description:
    Provides a list of the article_ids and the number of unique users that have
    interacted with the article using the user_item matrix, sorted by the number
    of unique users in descending order
    """
    article_counts = []
    for article_id in article_ids:
        if article_id in user_item.columns:
            # Count number of users who interacted with this article (sum of 1's in column)
            count = user_item[article_id].sum()
            article_counts.append([article_id, count])
    
    # Sort by count in descending order
    ranked_article_unique_counts = sorted(article_counts, key=lambda x: x[1], reverse=True)
    
    return ranked_article_unique_counts


def get_user_articles(user_id, user_item=user_item):
    """
    INPUT:
    user_id - (int) a user id
    user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    
    OUTPUT:
    article_ids - (list) a list of the article ids seen by the user
    article_names - (list) a list of article names associated with the list of article ids 
                    (this is identified by the title column in df)
    
    Description:
    Provides a list of the article_ids and article titles that have been seen by a user
    """
    # Get articles user has interacted with (where value is 1)
    user_articles = user_item.loc[user_id]
    article_ids = user_articles[user_articles == 1].index.tolist()
    
    # Get article names
    article_names = get_article_names(article_ids)
    
    return article_ids, article_names # return the ids and names


def user_user_recs(user_id, m=10):
    """
    INPUT:
    user_id - (int) a user id
    m - (int) the number of recommendations you want for the user
    
    OUTPUT:
    recs - (list) a list of recommendations for the user
    
    Description:
    Loops through the users based on closeness to the input user_id
    For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    Users who are the same closeness are chosen arbitrarily as the 'next' user
    
    For the user where the number of recommended articles starts below m 
    and ends exceeding m, the last items are chosen arbitrarily
    
    """
    # Get similar users
    similar_users = find_similar_users(user_id)
    
    # Get articles user has already seen
    user_articles, _ = get_user_articles(user_id)
    user_articles_set = set(user_articles)
    
    recs = []
    
    # Loop through similar users
    for similar_user in similar_users:
        if len(recs) >= m:
            break
            
        # Get articles this similar user has seen
        similar_user_articles, _ = get_user_articles(similar_user)
        
        # Find articles this user hasn't seen
        for article in similar_user_articles:
            if article not in user_articles_set and article not in recs:
                recs.append(article)
                if len(recs) >= m:
                    break
    
    return recs[:m] # return your recommendations for this user_id    

In [ ]:
# Check Results
get_article_names(user_user_recs(1, 10)) # Return 10 recommendations for user 1

In [ ]:
get_ranked_article_unique_counts([1320, 232, 844])

In [ ]:
# Test your functions here - No need to change this code - just run this cell
assert set(get_article_names([1024, 1176, 1305, 1314, 1422, 1427])) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_article_names([1320, 232, 844])) == set(['housing (2015): united states demographic measures','self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook']), "Oops! Your the get_article_names function doesn't work quite how we expect."
assert set(get_user_articles(20)[0]) == set([1320, 232, 844])
assert set(get_user_articles(20)[1]) == set(['housing (2015): united states demographic measures', 'self-service data preparation with ibm data refinery','use the cloudant-spark connector in python notebook'])
assert set(get_user_articles(2)[0]) == set([1024, 1176, 1305, 1314, 1422, 1427])
assert set(get_user_articles(2)[1]) == set(['using deep learning to reconstruct high-resolution audio', 'build a python app on the streaming analytics service', 'gosales transactions for naive bayes model', 'healthcare python streaming application demo', 'use r dataframes & ibm watson natural language understanding', 'use xgboost, scikit-learn & ibm watson machine learning apis'])
assert get_ranked_article_unique_counts([1320, 232, 844])[0] == [1320, 123], "Oops! Your the get_ranked_article_unique_counts function doesn't work quite how we expect.\nMake sure you are using the user_item matrix to create the article counts."
print("If this is all you see, you passed all of our tests!  Nice job!")

`4.` Now we are going to improve the consistency of the **user_user_recs** function from above.  

* Instead of arbitrarily choosing when we obtain users who are all the same closeness to a given user - choose the users that have the most total article interactions before choosing those with fewer article interactions.


* Instead of arbitrarily choosing articles from the user where the number of recommended articles starts below m and ends exceeding m, choose articles with the articles with the most total interactions before choosing those with fewer total interactions. This ranking should be  what would be obtained from the **top_articles** function you wrote earlier.

In [ ]:
def get_top_sorted_users(user_id, user_item=user_item):
    """
    INPUT:
    user_id - (int)
    user_item - (pandas dataframe) matrix of users by articles: 
            1's when a user has interacted with an article, 0 otherwise
    
            
    OUTPUT:
    neighbors_df - (pandas dataframe) a dataframe with:
                    neighbor_id - is a neighbor user_id
                    similarity - measure of the similarity of each user to the provided user_id
                    num_interactions - the number of articles viewed by the user
                    
    Other Details - sort the neighbors_df by the similarity and then by number of interactions where 
                    highest of each is higher in the dataframe, i.e. Descending order
     
    """
    # Find similar users with similarity scores
    similar_users_with_sim = find_similar_users(user_id, user_item=user_item, include_similarity=True)
    
    # Create lists for DataFrame
    neighbor_ids = []
    similarities = []
    num_interactions = []
    
    for user_sim_pair in similar_users_with_sim:
        neighbor_id = user_sim_pair[0]
        similarity = user_sim_pair[1]
        
        # Count interactions for this user (sum of 1's in their row)
        interactions = user_item.loc[neighbor_id].sum()
        
        neighbor_ids.append(neighbor_id)
        similarities.append(similarity)
        num_interactions.append(interactions)
    
    # Create DataFrame
    neighbors_df = pd.DataFrame({
        'neighbor_id': neighbor_ids,
        'similarity': similarities,
        'num_interactions': num_interactions
    })
    
    # Sort by similarity (descending) then by num_interactions (descending)
    neighbors_df = neighbors_df.sort_values(['similarity', 'num_interactions'], ascending=[False, False])
    
    return neighbors_df # Return the dataframe specified in the doc_string


def user_user_recs_part2(user_id, m=10):
    """
    INPUT:
    user_id - (int) a user id
    m - (int) the number of recommendations you want for the user
    
    OUTPUT:
    recs - (list) a list of recommendations for the user by article id
    rec_names - (list) a list of recommendations for the user by article title
    
    Description:
    Loops through the users based on closeness to the input user_id
    For each user - finds articles the user hasn't seen before and provides them as recs
    Does this until m recommendations are found
    
    Notes:
    * Choose the users that have the most total article interactions 
    before choosing those with fewer article interactions.

    * Choose articles with the articles with the most total interactions 
    before choosing those with fewer total interactions. 
   
    """
    # Get sorted similar users (by similarity then interactions)
    neighbors_df = get_top_sorted_users(user_id)
    
    # Get articles user has already seen
    user_articles, _ = get_user_articles(user_id)
    user_articles_set = set(user_articles)
    
    # Collect all potential recommendations from similar users
    potential_recs = []
    
    for _, neighbor_row in neighbors_df.iterrows():
        neighbor_id = neighbor_row['neighbor_id']
        
        # Get articles this neighbor has seen
        neighbor_articles, _ = get_user_articles(neighbor_id)
        
        # Add unseen articles to potential recommendations
        for article in neighbor_articles:
            if article not in user_articles_set and article not in potential_recs:
                potential_recs.append(article)
    
    # Rank potential recommendations by their popularity (total interactions)
    if potential_recs:
        ranked_recs = get_ranked_article_unique_counts(potential_recs)
        # Extract just the article IDs in order of popularity
        recs = [article_id for article_id, count in ranked_recs][:m]
    else:
        # Fallback to top articles if no recommendations found
        recs = get_top_article_ids(m)[:m]
    
    return recs, get_article_names(recs) # return your recommendations for this user_id

In [ ]:
# Quick spot check - don't change this code - just use it to test your functions
rec_ids, rec_names = user_user_recs_part2(20, 10)
print("The top 10 recommendations for user 20 are the following article ids:")
print(rec_ids)
print()
print("The top 10 recommendations for user 20 are the following article names:")
print(rec_names)

`5.` Use your functions from above to correctly fill in the solutions to the dictionary below.  Then test your dictionary against the solution.  Provide the code you need to answer each following the comments below.

In [ ]:
print(get_top_sorted_users(1, user_item=user_item).head(n=1))
print(get_top_sorted_users(2, user_item=user_item).head(n=10))
print(get_top_sorted_users(131, user_item=user_item).head(n=10))

In [ ]:
### Tests with a dictionary of results
user1_most_sim = get_top_sorted_users(1, user_item=user_item).iloc[0]['neighbor_id'] # Find the user that is most similar to user 1 
user2_6th_sim = get_top_sorted_users(2, user_item=user_item).iloc[5]['neighbor_id'] # Find the 6th most similar user to user 2
user131_10th_sim = get_top_sorted_users(131, user_item=user_item).iloc[9]['neighbor_id'] # Find the 10th most similar user to user 131

In [ ]:
## Dictionary Test Here
sol_5_dict = {
    'The user that is most similar to user 1.': user1_most_sim, 
    'The user that is the 6th most similar to user 2.': user2_6th_sim,
    'The user that is the 10th most similar to user 131.': user131_10th_sim,
}

t.sol_5_test(sol_5_dict)

`6.` If we were given a new user, which of the above functions would you be able to use to make recommendations?  Explain.  Can you think of a better way we might make recommendations?  Use the cell below to explain a better method for new users.

**Answer:**

For new users with no interaction history, we cannot use user-user collaborative filtering because there's no data to compute similarities with other users. 

The best approach for new users is to use **rank-based recommendations** - recommend the most popular articles that have the most interactions overall. This provides a reasonable starting point based on community preferences.

Once new users start interacting with articles, we can begin using collaborative filtering to find similar users and make personalized recommendations. This is known as the "cold start" problem in recommendation systems.

`7.` Using your existing functions, provide the top 10 recommended articles you would provide for the a new user below.  You can test your function against our thoughts to make sure we are all on the same page with how we might make a recommendation.

In [ ]:
# What would your recommendations be for this new user 0?  As a new user, they have no observed articles.
# Provide a list of the top 10 article ids you would give to 
new_user_recs = get_top_article_ids(10) # Your recommendations here

In [ ]:
assert set(new_user_recs) == {1314, 1429, 1293, 1427, 1162, 1364, 1304, 1170, 1431, 1330}, "Oops!  It makes sense that in this case we would want to recommend the most popular articles, because we don't know anything about these users."

print("That's right!  Nice job!")

### <a class="anchor" id="Content-Recs">Part IV: Content Based Recommendations</a>

Another method we might use to make recommendations is to recommend similar articles that are possibly related. One way we can find article relationships is by clustering text about those articles.  Let's consider content to be the article **title**, as it is the only text we have available. One point to highlight, there isn't one way to create a content based recommendation, especially considering that text information can be processed in many ways.  

`1.` Use the function bodies below to create a content based recommender function `make_content_recs`. We'll use TF-IDF to create a matrix based off article titles, and use this matrix to create clusters of related articles. You can use this function to make recommendations of new articles.

In [ ]:
df.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
# unique articles
df_unique_articles = df[['article_id', 'title']].drop_duplicates()

In [ ]:
# Create a vectorizer using TfidfVectorizer and fit it to the article titles
max_features = 200
max_df = 0.75
min_df = 5

vectorizer = TfidfVectorizer(
    max_df=max_df,
    min_df=min_df,
    stop_words="english",
    max_features=max_features,
)
print("Running TF-IDF")
X_tfidf = vectorizer.fit_transform(df_unique_articles['title']) # Fit the vectorizer to the article titles

print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

lsa = make_pipeline(TruncatedSVD(n_components=50), Normalizer(copy=False))
X_lsa = lsa.fit_transform(X_tfidf) # Fit the LSA model to the vectorized article titles
explained_variance = lsa[0].explained_variance_ratio_.sum()

print(f"Explained variance of the SVD step: {explained_variance * 100:.1f}%")

In [ ]:
# Let's map the inertia for different number of clusters to find the optimal number of clusters
# We'll plot it to see the elbow
inertia = []
clusters = 300
step = 25
max_iter = 50
n_init = 5
random_state = 42
for k in range(1, clusters, step):
    kmeans = KMeans(
        n_clusters=k,
        max_iter=max_iter,
        n_init=n_init,
        random_state=random_state,
    ).fit(X_lsa)
    # inertia is the sum of squared distances to the closest cluster center
    inertia.append(kmeans.inertia_)
plt.plot(range(1, clusters, step), inertia)
plt.xlabel('Number of clusters')

There appears to be an elbow about 50, so we'll use 50 clusters.

In [ ]:
n_clusters = 50 # Number of clusters
kmeans = KMeans(
    n_clusters=n_clusters,
    max_iter=max_iter,
    n_init=n_init,
    random_state=random_state,
).fit(X_lsa)

In [ ]:
# create a new column `title_cluster` and assign it the kmeans cluster labels
# First we need to map the labels to df_unique_articles article ids and then apply those to df
article_cluster_map = dict(zip(df_unique_articles['article_id'], kmeans.labels_)) # Map article_id to cluster label
df['title_cluster'] = df['article_id'].map(article_cluster_map) # apply map to create title clusters

In [ ]:
# Let's check the number of articles in each cluster
np.array(np.unique(kmeans.labels_, return_counts=True)).T

In [ ]:
def get_similar_articles(article_id, df=df):
    """
    INPUT:
    article_id - (int) an article id 
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    article_ids - (list) a list of article ids that are in the same title cluster
    
    Description:
    Returns a list of the article ids that are in the same title cluster
    """
    # Get the cluster for the input article
    title_cluster = df[df['article_id'] == article_id]['title_cluster'].iloc[0]
    
    # Get all articles in the same cluster
    articles_in_cluster = df[df['title_cluster'] == title_cluster]['article_id'].unique().tolist()
    
    # remove the input article_id from the list
    articles_in_cluster = [aid for aid in articles_in_cluster if aid != article_id]
    
    return articles_in_cluster

In [ ]:
def make_content_recs(article_id, n, df=df):
    """
    INPUT:
    article_id - (int) an article id
    n - (int) the number of recommendations you want similar to the article id
    df - (pandas dataframe) df as defined at the top of the notebook
    
    OUTPUT:
    n_ranked_similar_articles - (list) a list of article ids that are in the same title cluster ranked
                                by popularity
    n_ranked_article_names - (list) a list of article names associated with the list of article ids
    
    Description:
    Returns a list of the n most ranked similar articles to a given article_id based on the title
    cluster in df. Rank similar articles using the function get_ranked_article_unique_counts.
    """
    # Get similar articles in the same cluster
    similar_articles = get_similar_articles(article_id, df)
    
    # Rank them by popularity using our existing function
    if similar_articles:
        ranked_articles = get_ranked_article_unique_counts(similar_articles, user_item)
        # Extract article IDs and limit to n
        n_ranked_similar_articles = [article_id for article_id, count in ranked_articles][:n]
        # Get article names
        n_ranked_article_names = get_article_names(n_ranked_similar_articles, df)
    else:
        # If no similar articles found, return empty lists
        n_ranked_similar_articles = []
        n_ranked_article_names = []
    
    return n_ranked_similar_articles, n_ranked_article_names
        

In [ ]:
# Test out your content recommendations given artice_id 25
rec_article_ids, rec_article_titles = make_content_recs(25, 10)
print(rec_article_ids)
print(rec_article_titles)

In [ ]:
assert len({1025, 593, 349, 821, 464, 29, 1042, 693, 524, 352}.intersection(set(rec_article_ids))) > 0, "Oops! Your the make_content_recs function doesn't work quite how we expect."

`2.` Now that you have put together your content-based recommendation system, use the cell below to write a summary explaining how your content based recommender works.  Do you see any possible improvements that could be made to your function? What other text data would be useful to help make better recommendations besides the article title?

**Content-Based Recommendation System Explanation:**

Our content-based recommendation system works by analyzing the textual similarity between article titles using the following approach:

1. **Text Preprocessing**: We use TF-IDF (Term Frequency-Inverse Document Frequency) vectorization to convert article titles into numerical feature vectors, removing common English stop words and limiting to 200 most important features.

2. **Dimensionality Reduction**: We apply Truncated SVD (Latent Semantic Analysis) to reduce the feature space to 50 components, capturing the most important semantic patterns while reducing noise.

3. **Clustering**: K-means clustering with 50 clusters groups articles with similar content themes together based on their reduced feature representations.

4. **Recommendation Generation**: For a given article, we find all articles in the same cluster and rank them by popularity (number of user interactions).

**Possible Improvements:**
- **More Text Data**: Using article abstracts, descriptions, or full content would provide richer semantic understanding
- **Advanced NLP**: Techniques like word embeddings (Word2Vec, GloVe) or transformer models (BERT) could capture deeper semantic relationships
- **Hybrid Approach**: Combining content similarity with collaborative filtering scores for better recommendations
- **Dynamic Clustering**: Periodically retraining clusters as new articles are added
- **User Context**: Incorporating user preferences and reading history to personalize content recommendations

**Additional Text Data That Would Help:**
- Article abstracts or summaries
- Article tags/categories
- Author information
- Full article content
- User comments or reviews
- Related topics or keywords

### <a class="anchor" id="Matrix-Fact">Part V: Matrix Factorization</a>

In this part of the notebook, you will build use matrix factorization to make article recommendations to users.

`1.` You should have already created a **user_item** matrix above in **question 1** of **Part III** above.  This first question here will just require that you run the cells to get things set up for the rest of **Part V** of the notebook. 

In [ ]:
# quick look at the matrix
user_item.head()

`2.` In this situation, you can use Singular Value Decomposition from [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html) on the user-item matrix.  Use the cell to perform SVD.

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import precision_score, recall_score, accuracy_score
# Using the full number of components which equals the number of columns
svd = TruncatedSVD(n_components=len(user_item.columns), n_iter=5, random_state=42)

u = svd.fit_transform(user_item)
v = svd.components_
s = svd.singular_values_ 
print('u', u.shape)
print('s', s.shape)
print('vt', v.shape)

`3.` Now for the tricky part, how do we choose the number of latent features to use?  Running the below cell, you can see that as the number of latent features increases, we obtain better metrics when making predictions for the 1 and 0 values in the user-item matrix.  Run the cell below to get an idea of how our metrics improve as we increase the number of latent features.

In [ ]:
num_latent_feats = np.arange(10, 700+10, 20)
metric_scores = []

for k in num_latent_feats:
    # restructure with k latent features
    u_new, vt_new = u[:, :k], v[:k, :]
    
    # take dot product
    user_item_est = abs(np.around(np.dot(u_new, vt_new))).astype(int)
    # make sure the values are between 0 and 1
    user_item_est = np.clip(user_item_est, 0, 1)
    
    # total errors and keep track of them
    acc = accuracy_score(user_item.values.flatten(), user_item_est.flatten())
    precision = precision_score(user_item.values.flatten(), user_item_est.flatten())
    recall = recall_score(user_item.values.flatten(), user_item_est.flatten())
    metric_scores.append([acc, precision, recall])
    
    
plt.plot(num_latent_feats, metric_scores, label=['Accuracy', 'Precision', 'Recall'])
plt.legend()
plt.xlabel('Number of Latent Features')
plt.title('Metrics vs. Number of Latent Features')

`4.` From the above, we can't really be sure how many features to use, because simply having a better way to predict the 1's and 0's of the matrix doesn't exactly give us an indication of if we are able to make good recommendations. Given the plot above, what would you pick for the number of latent features and why?

**Analysis of Optimal Latent Features:**

Based on the metrics plot, I would choose approximately **200 latent features** for the following reasons:

1. **Performance Plateau**: After around 150-200 features, the improvement in all metrics (accuracy, precision, recall) starts to plateau, with diminishing returns for additional features.

2. **Computational Efficiency**: Using fewer features (200 vs 700) significantly reduces computational complexity while maintaining good performance.

3. **Overfitting Prevention**: Too many latent features (approaching the full 714) might lead to overfitting, where the model memorizes the training data but doesn't generalize well.

4. **Balanced Performance**: At 200 features, we achieve a good balance between all three metrics without heavily favoring one over the others.

5. **Practical Considerations**: 200 features capture the most important patterns in user-article interactions while remaining interpretable and computationally manageable.

The goal is to find the "elbow" in the performance curve where additional complexity doesn't provide proportional benefits. Around 200 features appears to be this optimal point where we maximize recommendation quality while minimizing computational overhead.

`5.` Using 200 latent features and the values of U, S, and V transpose we calculated above, create an article id recommendation function that finds similar article ids to the one provide.

Create a list of 10 recommendations that are similar to article with id 4.  The function should provide these recommendations by finding articles that have the most similar latent features as the provided article.

In [ ]:
def get_svd_similar_article_ids(article_id, vt, user_item=user_item, include_similarity=False):
    """
    INPUT:
    article_id - (int) an article id
    vt - (numpy array) vt matrix from SVD
    user_item - (pandas dataframe) matrix of users by articles: 
                1's when a user has interacted with an article, 0 otherwise
    include_similarity - (bool) whether to include the similarity in the output
    
    OUTPUT:
    article_ids - (list) a list of article ids that are similar based on SVD factors
    
    Description:
    Returns a list of the article ids similar using SVD factorization
    """
    # Find the index of the article_id in user_item columns
    article_idx = user_item.columns.tolist().index(article_id)
    
    # Find the cosine similarity of all articles
    # Transpose vt to get a 714 x k matrix for cosine similarity
    cos_sim = cosine_similarity(vt.T)
    
    # Get similarities only for the article_idx
    similarities = cos_sim[article_idx]
    
    # Create pairs of (article_id, similarity) and sort by similarity
    article_sim_pairs = list(zip(user_item.columns, similarities))
    article_sim_pairs.sort(key=lambda x: x[1], reverse=True)
    
    # Remove the input article itself
    article_sim_pairs = [pair for pair in article_sim_pairs if pair[0] != article_id]
    
    if include_similarity:
        return [[int(aid), sim] for aid, sim in article_sim_pairs] # return a list of lists with [[similar_article_id, similarity], ...]
    
    most_similar_items = [int(aid) for aid, sim in article_sim_pairs]
    return most_similar_items

In [ ]:
# Create a vt_new matrix with 200 latent features
k = 200
vt_new = v[:k, :]

In [ ]:
# What is the article name for article_id 4?
print("Current article:", get_article_names([4], df=df)[0])

In [ ]:
# What are the top 10 most similar articles to article_id 4?
rec_articles = get_svd_similar_article_ids(4, vt_new, user_item=user_item)[:10]
rec_articles

In [ ]:
# What are the top 10 most similar articles to article_id 4?
get_article_names(rec_articles, df=df)

In [ ]:
assert set(rec_articles) == {1199, 1068, 486, 1202, 176, 1120, 244, 793, 58, 132}, "Oops! Your the get_svd_similar_article_ids function doesn't work quite how we expect."
print("That's right!  Great job!")

`6.` Use the cell below to comment on the results you found in the previous question. Given the circumstances of your results, discuss what you might do to determine if the recommendations you make above are an improvement to how users currently find articles, either by Sections 2, 3, or 4? Add any tradeoffs between each of the methods, and how you could leverage each type for different situations including new users with no history, recently new users with little history, and users with a lot of history. 

**Analysis of SVD Matrix Factorization Results:**

The SVD-based recommendations for article 4 show promising results by identifying articles with similar latent feature patterns. However, to evaluate whether these recommendations are truly better than our previous methods, we need to consider several factors:

**Comparison of Recommendation Methods:**

1. **Rank-Based (Section 2)**: 
   - Pros: Simple, works for all users including new ones, based on proven popularity
   - Cons: No personalization, ignores user preferences, same recommendations for everyone

2. **User-User Collaborative Filtering (Section 3)**:
   - Pros: Personalized based on similar users, leverages community wisdom
   - Cons: Cold start problem, sparse data issues, computationally expensive

3. **Content-Based (Section 4)**:
   - Pros: Works with article features, no cold start for new articles, explainable
   - Cons: Limited by available text data, may create filter bubbles, requires domain knowledge

4. **Matrix Factorization (Section 5)**:
   - Pros: Handles sparsity well, captures latent relationships, scalable
   - Cons: Less interpretable, requires tuning, cold start for new users/items

**Evaluation Strategy:**

To determine which method works best, I would implement:

1. **A/B Testing**: Deploy different methods to user segments and measure engagement metrics
2. **Offline Evaluation**: Use train/test splits to measure precision@k, recall@k, and NDCG
3. **User Studies**: Survey users about recommendation relevance and satisfaction
4. **Business Metrics**: Track click-through rates, time spent reading, and user retention

**Recommendation Strategy by User Type:**

- **New Users (no history)**: Use rank-based recommendations to provide popular, high-quality content
- **Users with little history (1-5 interactions)**: Combine rank-based with content-based recommendations
- **Users with moderate history (5-20 interactions)**: Use collaborative filtering with content-based fallbacks
- **Power Users (20+ interactions)**: Use matrix factorization for sophisticated personalization

**Testing Recommendation Quality:**

1. **Precision/Recall**: Measure how many recommendations users actually interact with
2. **Diversity**: Ensure recommendations aren't too narrow or repetitive  
3. **Novelty**: Balance between safe popular choices and discovering new content
4. **User Feedback**: Implement thumbs up/down to continuously improve
5. **Long-term Engagement**: Track whether recommendations increase overall platform usage

The best approach would likely be a **hybrid system** that combines multiple methods based on data availability and user characteristics.

<a id='conclusions'></a>
### Extras
Using your workbook, you could now save your recommendations for each user, develop a class to make new predictions and update your results, and make a flask app to deploy your results.  These tasks are beyond what is required for this project.  However, from what you learned in the lessons, you certainly capable of taking these tasks on to improve upon your work here!


## Conclusion

> Congratulations!  You have reached the end of the Recommendation Systems project! 

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the [rubric](https://review.udacity.com/#!/rubrics/2322/view). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.


## Directions to Submit

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations! 

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Recommendations_with_IBM.ipynb'])